In [1]:
# import libraries
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm
import re

In [2]:
nppes = pd.read_csv('data/NPPES_Data_Dissemination_February_2022/npidata_pfile_20050523-20220213_FileHeader.csv')
nppes.columns

Index(['NPI', 'Entity Type Code', 'Replacement NPI',
       'Employer Identification Number (EIN)',
       'Provider Organization Name (Legal Business Name)',
       'Provider Last Name (Legal Name)', 'Provider First Name',
       'Provider Middle Name', 'Provider Name Prefix Text',
       'Provider Name Suffix Text',
       ...
       'Healthcare Provider Taxonomy Group_7',
       'Healthcare Provider Taxonomy Group_8',
       'Healthcare Provider Taxonomy Group_9',
       'Healthcare Provider Taxonomy Group_10',
       'Healthcare Provider Taxonomy Group_11',
       'Healthcare Provider Taxonomy Group_12',
       'Healthcare Provider Taxonomy Group_13',
       'Healthcare Provider Taxonomy Group_14',
       'Healthcare Provider Taxonomy Group_15', 'Certification Date'],
      dtype='object', length=330)

In [3]:
# columns to keep
referral_from_cols = ['referral_id', 'from_npi']
referral_to_cols = ['referral_id', 'to_npi']
referrals_cols = ['referral_id',
             'patient_count', 
             'transaction_count', 
             'average_day_wait',
             'std_day_wait']
profile_cols = ['NPI',
               'Entity Type Code',
                'Provider Organization Name (Legal Business Name)',
                'Provider Last Name (Legal Name)',
                'Provider First Name',
                'Provider Middle Name', 
                'Provider Name Prefix Text',
                'Provider Name Suffix Text',
                'Provider Credential Text',
                'Provider First Line Business Practice Location Address',
                'Provider Second Line Business Practice Location Address',
                'Provider Business Practice Location Address City Name',
                'Provider Business Practice Location Address State Name',
                'Provider Business Practice Location Address Postal Code'
               ]
taxonomy_code_cols = [col for col in nppes.columns if 'Healthcare Provider Taxonomy Code' in col ]
taxonomy_switch_cols = [col for col in nppes.columns if 'Healthcare Provider Primary Taxonomy Switch' in col]
taxonomy_cols = ['NPI']+ [col for col in nppes.columns 
                 if 'Healthcare Provider Taxonomy Code'in col 
                 or 'Healthcare Provider Primary Taxonomy Switch' in col]

In [4]:
chunks = pd.read_csv('data/DocGraph_Hop_Teaming_2018_Commercial/DocGraph_Hop_Teaming_2018.csv', chunksize=1000)
df= next(chunks)

In [72]:
chunks = pd.read_csv('data/NPPES_Data_Dissemination_February_2022/npidata_pfile_20050523-20220213.csv', chunksize=1000)
df = next(chunks)

In [74]:
df.dropna(subset= ['NPI','Provider Business Practice Location Address Postal Code'], inplace = True)
df

,NPI,Entity Type Code,Replacement NPI,Employer Identification Number (EIN),Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,...,Healthcare Provider Taxonomy Group_7,Healthcare Provider Taxonomy Group_8,Healthcare Provider Taxonomy Group_9,Healthcare Provider Taxonomy Group_10,Healthcare Provider Taxonomy Group_11,Healthcare Provider Taxonomy Group_12,Healthcare Provider Taxonomy Group_13,Healthcare Provider Taxonomy Group_14,Healthcare Provider Taxonomy Group_15,Certification Date
0,1679576722,1.0,NaN,NaN,NaN,WIEBE,DAVID,A,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1588667638,1.0,NaN,NaN,NaN,PILCHER,WILLIAM,C,DR.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1497758544,2.0,NaN,<UNAVAIL>,"CUMBERLAND COUNTY HOSPITAL SYSTEM, INC",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1215930367,1.0,NaN,NaN,NaN,GRESSOT,LAURENT,NaN,DR.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1023011178,2.0,NaN,<UNAVAIL>,COLLABRIA CARE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06/09/2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1417950635,1.0,NaN,NaN,NaN,MARINO,CHRIS,J,DR.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,1326041542,1.0,NaN,NaN,NaN,TAFEL,ALLEN,C,DR.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,1235132457,1.0,NaN,NaN,NaN,ADAMSON,BRENT,E,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,1144223363,1.0,NaN,NaN,NaN,KATZ,MARC,A,DR.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
# extract 5-digit zip code
zip_codes = df['Provider Business Practice Location Address Postal Code'].astype(int).astype(str)
correct_zip_code = []
for zip_code in zip_codes:
    if len(zip_code) < 5:
        correct_zip_code.append(zip_code.zfill(5))
    elif len(zip_code) == 5:
        correct_zip_code.append(zip_code)
    elif len(zip_code) < 9:
        correct_zip_code.append(zip_code.zfill(9)[:5])
    else :
        correct_zip_code.append(zip_code[:5])

In [77]:
df['Provider Business Practice Location Address Postal Code']=correct_zip_code

In [79]:
df

,NPI,Entity Type Code,Replacement NPI,Employer Identification Number (EIN),Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,...,Healthcare Provider Taxonomy Group_7,Healthcare Provider Taxonomy Group_8,Healthcare Provider Taxonomy Group_9,Healthcare Provider Taxonomy Group_10,Healthcare Provider Taxonomy Group_11,Healthcare Provider Taxonomy Group_12,Healthcare Provider Taxonomy Group_13,Healthcare Provider Taxonomy Group_14,Healthcare Provider Taxonomy Group_15,Certification Date
0,1679576722,1.0,NaN,NaN,NaN,WIEBE,DAVID,A,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1588667638,1.0,NaN,NaN,NaN,PILCHER,WILLIAM,C,DR.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1497758544,2.0,NaN,<UNAVAIL>,"CUMBERLAND COUNTY HOSPITAL SYSTEM, INC",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1215930367,1.0,NaN,NaN,NaN,GRESSOT,LAURENT,NaN,DR.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1023011178,2.0,NaN,<UNAVAIL>,COLLABRIA CARE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06/09/2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1417950635,1.0,NaN,NaN,NaN,MARINO,CHRIS,J,DR.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,1326041542,1.0,NaN,NaN,NaN,TAFEL,ALLEN,C,DR.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,1235132457,1.0,NaN,NaN,NaN,ADAMSON,BRENT,E,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,1144223363,1.0,NaN,NaN,NaN,KATZ,MARC,A,DR.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [160]:
taxonomy_code = df.loc[:, ['NPI'] +taxonomy_code_cols]
taxonomy_code = pd.melt(taxonomy_code, id_vars = 'NPI', value_vars = taxonomy_code_cols)
taxonomy_code['match_num']=taxonomy_code['variable'].str.extractall('(\d+)').unstack()
taxonomy_code = taxonomy_code.rename(columns={'value':'taxonomy_code'})
taxonomy_code

,NPI,variable,taxonomy_code,match_num
0,1679576722,Healthcare Provider Taxonomy Code_1,207X00000X,1
1,1588667638,Healthcare Provider Taxonomy Code_1,207RC0000X,1
2,1497758544,Healthcare Provider Taxonomy Code_1,251G00000X,1
3,1306849450,Healthcare Provider Taxonomy Code_1,NaN,1
4,1215930367,Healthcare Provider Taxonomy Code_1,174400000X,1
...,...,...,...,...
14995,1417950635,Healthcare Provider Taxonomy Code_15,NaN,15
14996,1326041542,Healthcare Provider Taxonomy Code_15,NaN,15
14997,1235132457,Healthcare Provider Taxonomy Code_15,NaN,15
14998,1144223363,Healthcare Provider Taxonomy Code_15,NaN,15


In [161]:
taxonomy_switch = df.loc[:, ['NPI']+taxonomy_switch_cols]
taxonomy_switch = pd.melt(taxonomy_switch, id_vars = 'NPI', value_vars = taxonomy_switch_cols)
taxonomy_switch = taxonomy_switch[taxonomy_switch['value']=='Y']
taxonomy_switch['match_num'] = taxonomy_switch['variable'].str.extractall('(\d+)').unstack()
taxonomy_switch

,NPI,variable,value,match_num
0,1679576722,Healthcare Provider Primary Taxonomy Switch_1,Y,1
1,1588667638,Healthcare Provider Primary Taxonomy Switch_1,Y,1
2,1497758544,Healthcare Provider Primary Taxonomy Switch_1,Y,1
5,1023011178,Healthcare Provider Primary Taxonomy Switch_1,Y,1
7,1841293990,Healthcare Provider Primary Taxonomy Switch_1,Y,1
...,...,...,...,...
6426,1699778829,Healthcare Provider Primary Taxonomy Switch_7,Y,7
7942,1811990047,Healthcare Provider Primary Taxonomy Switch_8,Y,8
8714,1164425336,Healthcare Provider Primary Taxonomy Switch_9,Y,9
8964,1619970845,Healthcare Provider Primary Taxonomy Switch_9,Y,9


In [163]:
df = (pd.merge(taxonomy_code, taxonomy_switch, how = 'inner', on = ['NPI', 'match_num']).
 drop(columns=['variable_x','variable_y', 'match_num','value'])).merge(df.loc[:,profile_cols], on ='NPI')


,NPI,taxonomy_code,Entity Type Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code
0,1679576722,207X00000X,1.0,NaN,WIEBE,DAVID,A,NaN,NaN,M.D.,3500 CENTRAL AVE,NaN,KEARNEY,NE,688472944.0
1,1588667638,207RC0000X,1.0,NaN,PILCHER,WILLIAM,C,DR.,NaN,MD,1824 KING STREET,SUITE 300,JACKSONVILLE,FL,322044736.0
2,1497758544,251G00000X,2.0,"CUMBERLAND COUNTY HOSPITAL SYSTEM, INC",NaN,NaN,NaN,NaN,NaN,NaN,3418 VILLAGE DR,NaN,FAYETTEVILLE,NC,283044552.0
3,1023011178,251G00000X,2.0,COLLABRIA CARE,NaN,NaN,NaN,NaN,NaN,NaN,414 S JEFFERSON ST,NaN,NAPA,CA,945594515.0
4,1841293990,231H00000X,1.0,NaN,WORTSMAN,SUSAN,NaN,NaN,NaN,MA-CCC,425 E 25TH ST,NaN,NEW YORK,NY,100102547.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
927,1699778829,207X00000X,2.0,CHIPPEWA VALLEY ORTHOPEDICS AND SPORTS MEDICIN...,NaN,NaN,NaN,NaN,NaN,NaN,1200 OAKLEAF WAY,SUITE A,ALTOONA,WI,547202245.0
928,1811990047,333600000X,2.0,NBN INFUSIONS INC,NaN,NaN,NaN,NaN,NaN,NaN,2 PIN OAK LN,UNIT 250,CHERRY HILL,NJ,80031632.0
929,1164425336,3336H0001X,2.0,"BIOSCRIP PHARMACY(NY), INC",NaN,NaN,NaN,NaN,NaN,NaN,1 VERMONT DR.,NaN,LAKE SUCCESS,NY,11042.0
930,1619970845,3336H0001X,2.0,"BIOSCRIP PHARMACY SERVICES, INC",NaN,NaN,NaN,NaN,NaN,NaN,5700 PERIMETER DR STE B,NaN,DUBLIN,OH,430173247.0


In [166]:
df.columns = [x.lower().replace(' ', '_') for x in df.columns]
df

,npi,taxonomy_code,entity_type_code,provider_organization_name_(legal_business_name),provider_last_name_(legal_name),provider_first_name,provider_middle_name,provider_name_prefix_text,provider_name_suffix_text,provider_credential_text,provider_first_line_business_practice_location_address,provider_second_line_business_practice_location_address,provider_business_practice_location_address_city_name,provider_business_practice_location_address_state_name,provider_business_practice_location_address_postal_code
0,1679576722,207X00000X,1.0,NaN,WIEBE,DAVID,A,NaN,NaN,M.D.,3500 CENTRAL AVE,NaN,KEARNEY,NE,688472944.0
1,1588667638,207RC0000X,1.0,NaN,PILCHER,WILLIAM,C,DR.,NaN,MD,1824 KING STREET,SUITE 300,JACKSONVILLE,FL,322044736.0
2,1497758544,251G00000X,2.0,"CUMBERLAND COUNTY HOSPITAL SYSTEM, INC",NaN,NaN,NaN,NaN,NaN,NaN,3418 VILLAGE DR,NaN,FAYETTEVILLE,NC,283044552.0
3,1023011178,251G00000X,2.0,COLLABRIA CARE,NaN,NaN,NaN,NaN,NaN,NaN,414 S JEFFERSON ST,NaN,NAPA,CA,945594515.0
4,1841293990,231H00000X,1.0,NaN,WORTSMAN,SUSAN,NaN,NaN,NaN,MA-CCC,425 E 25TH ST,NaN,NEW YORK,NY,100102547.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
927,1699778829,207X00000X,2.0,CHIPPEWA VALLEY ORTHOPEDICS AND SPORTS MEDICIN...,NaN,NaN,NaN,NaN,NaN,NaN,1200 OAKLEAF WAY,SUITE A,ALTOONA,WI,547202245.0
928,1811990047,333600000X,2.0,NBN INFUSIONS INC,NaN,NaN,NaN,NaN,NaN,NaN,2 PIN OAK LN,UNIT 250,CHERRY HILL,NJ,80031632.0
929,1164425336,3336H0001X,2.0,"BIOSCRIP PHARMACY(NY), INC",NaN,NaN,NaN,NaN,NaN,NaN,1 VERMONT DR.,NaN,LAKE SUCCESS,NY,11042.0
930,1619970845,3336H0001X,2.0,"BIOSCRIP PHARMACY SERVICES, INC",NaN,NaN,NaN,NaN,NaN,NaN,5700 PERIMETER DR STE B,NaN,DUBLIN,OH,430173247.0


In [51]:
(df.
    reset_index().
    rename(columns = {'index':'referral_id'}).
    loc[:,referrals_cols])

,referral_id,patient_count,transaction_count,average_day_wait,std_day_wait
0,0,350,370,53.922,72.612
1,1,25,25,49.800,55.006
2,2,16,16,109.500,70.593
3,3,14,14,103.357,75.483
4,4,20,21,89.952,89.880
...,...,...,...,...,...
995,995,13,13,131.154,118.476
996,996,17,18,61.222,53.114
997,997,15,16,107.500,80.602
998,998,18,18,68.667,53.639


In [53]:
# create a reference_id and write hop_teaming_2018 csv to three tables in hop_teaming db
db = sqlite3.connect('data/hopteam.sqlite')
for chunk in tqdm(pd.read_csv('data/DocGraph_Hop_Teaming_2018_Commercial/DocGraph_Hop_Teaming_2018.csv', chunksize = 10000)):
    chunk = chunk.reset_index().rename(columns = {'index':'referral_id'}) # create a unique referral_id for each referal
    chunk.loc[:,referral_from_cols].to_sql('referral_from', db, if_exists = 'append', index = False) 
    chunk.loc[:,referral_to_cols].to_sql('referral_to', db, if_exists = 'append', index = False)
    chunk.loc[:,referrals_cols].to_sql('referrals', db, if_exists = 'append', index = False)
    db.close()    

0it [00:00, ?it/s]

In [ ]:
db = sqlite3.connect('data/hopteam.sqlite')
db.execute('CREATE INDEX from_npi ON hop_teaming(from_npi)')
db.close()

In [ ]:
db = sqlite3.connect('data/hopteam.sqlite')
tqdm(db.execute('CREATE INDEX to_npi ON hop_teaming(to_npi)'))
db.close()

In [12]:
db = sqlite3.connect('data/hopteam.sqlite')
tqdm(db.execute('CREATE INDEX transaction_count ON hop_teaming(transaction_count)'))
db.close()

0it [00:00, ?it/s]

In [ ]:
db = sqlite3.connect('data/hopteam.sqlite')
db.execute('CREATE INDEX average_day_wait ON hop_teaming(average_day_wait)')
db.close()

In [ ]:
# get taxonomy code for each npi and select 15 columns NPPES table to write into the db
db = sqlite3.connect('data/hopteam.sqlite')
for chunk in tqdm(pd.read_csv('data/NPPES_Data_Dissemination_February_2022/npidata_pfile_20050523-20220213.csv', chunksize=10000)):
    # drop NaN for NPI and Zip Code columns
    chunk.dropna(subset= ['NPI','Provider Business Practice Location Address Postal Code'], inplace = True)
    # melt columns of taxonomy codes
    taxonomy_code = chunk.loc[:, ['NPI'] +taxonomy_code_cols]
    taxonomy_code = pd.melt(taxonomy_code, id_vars = 'NPI', value_vars = taxonomy_code_cols)
    taxonomy_code['match_num']=taxonomy_code['variable'].str.extractall('(\d+)').unstack()
    taxonomy_code = taxonomy_code.rename(columns={'value':'taxonomy_code'})
    # melt columns of taxonomy switchs
    taxonomy_switch = chunk.loc[:, ['NPI']+taxonomy_switch_cols]
    taxonomy_switch = pd.melt(taxonomy_switch, id_vars = 'NPI', value_vars = taxonomy_switch_cols)
    taxonomy_switch = taxonomy_switch[taxonomy_switch['value']=='Y']
    taxonomy_switch['match_num'] = taxonomy_switch['variable'].str.extractall('(\d+)').unstack()
    # inner join the taxonomy codes table with taxonomy switch table and other profile columns
    chunk = (pd.merge(taxonomy_code, taxonomy_switch, how = 'inner', on = ['NPI', 'match_num']).
     drop(columns=['variable_x','variable_y', 'match_num','value'])).merge(df.loc[:,profile_cols], on ='NPI')
    # extract 5-digit zip codes and assign them to the Business Practice Zip Code column
    zip_codes = chunk['Provider Business Practice Location Address Postal Code'].astype(int).astype(str)
    correct_zip_code = []
    for zip_code in zip_codes:
        if len(zip_code) < 5:
            correct_zip_code.append(zip_code.zfill(5))
        elif len(zip_code) == 5:
            correct_zip_code.append(zip_code)
        elif len(zip_code) < 9:
            correct_zip_code.append(zip_code.zfill(9)[:5])
        else :
            correct_zip_code.append(zip_code[:5])
    chunk['Provider Business Practice Location Address Postal Code']=correct_zip_code
    # change the column names to lower cases and replace space with underscore
    chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]
    # write table to db
    chunk.to_sql('profile', db, if_exists = 'append', index = False)
db.close()

In [2]:
# Zip_CBSA
pd.read_excel('data/NPPES_Data_Dissemination_February_2022/ZIP_TRACT_122021.xlsx')

,zip,tract,usps_zip_pref_city,usps_zip_pref_state,res_ratio,bus_ratio,oth_ratio,tot_ratio
0,683,72023830102,SAN GERMAN,PR,0.000791,0.001116,0.000000,0.000800
1,683,72125840700,SAN GERMAN,PR,0.186219,0.370536,0.381643,0.201179
2,683,72125840400,SAN GERMAN,PR,0.300451,0.187500,0.115942,0.290308
3,683,72125840600,SAN GERMAN,PR,0.095325,0.007812,0.000000,0.088184
4,683,72121960300,SAN GERMAN,PR,0.042402,0.002232,0.019324,0.039435
...,...,...,...,...,...,...,...,...
172172,99163,53075000500,PULLMAN,WA,0.019942,0.001299,0.006263,0.018026
172173,99163,53075000200,PULLMAN,WA,0.205082,0.115584,0.284621,0.207639
172174,99163,53075001000,PULLMAN,WA,0.001415,0.000000,0.000000,0.001239
172175,78635,48171950100,HYE,TX,0.016667,0.000000,0.000000,0.015625


In [3]:
pd.read_csv('data/NPPES_Data_Dissemination_February_2022/nucc_taxonomy_220.csv')

,Code,Grouping,Classification,Specialization,Definition,Notes,Display Name,Section
0,193200000X,Group,Multi-Specialty,NaN,A business group of one or more individual pra...,[7/1/2003: new],Multi-Specialty Group,Individual
1,193400000X,Group,Single Specialty,NaN,A business group of one or more individual pra...,[7/1/2003: new],Single Specialty Group,Individual
2,207K00000X,Allopathic & Osteopathic Physicians,Allergy & Immunology,NaN,An allergist-immunologist is trained in evalua...,"Source: American Board of Medical Specialties,...",Allergy & Immunology Physician,Individual
3,207KA0200X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Allergy,Definition to come...,NaN,Allergy Physician,Individual
4,207KI0005X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Clinical & Laboratory Immunology,Definition to come...,NaN,Clinical & Laboratory Immunology (Allergy & Im...,Individual
...,...,...,...,...,...,...,...,...
863,343800000X,Transportation Services,Secured Medical Transport (VAN),NaN,A public or privately owned transportation ser...,NaN,Secured Medical Transport (VAN),Non-Individual
864,344600000X,Transportation Services,Taxi,NaN,A land commercial vehicle used for the transpo...,NaN,Taxi,Non-Individual
865,347D00000X,Transportation Services,Train,NaN,An organization or business licensed to provid...,NaN,Train,Non-Individual
866,347E00000X,Transportation Services,Transportation Broker,NaN,An organization that provides transportation f...,Source: Section 6083 of the Deficit Reduction ...,Transportation Broker,Non-Individual


In [57]:
query = """
SELECT * 
FROM referrals
LIMIT 5
"""
with sqlite3.connect('data/hopteam.sqlite') as db:
    ref_sqlite = pd.read_sql(query, db)
ref_sqlite

,referral_id,patient_count,transaction_count,average_day_wait,std_day_wait
0,0,350,370,53.922,72.612
1,1,25,25,49.800,55.006
2,2,16,16,109.500,70.593
3,3,14,14,103.357,75.483
4,4,20,21,89.952,89.880


In [83]:
query = """
SELECT *
FROM hop_teaming
WHERE transaction_count >= 50
    AND average_day_wait <50
    AND from_npi IN (SELECT npi FROM nppes WHERE entity_type_code = 1.0)
    AND to_npi IN (SELECT npi FROM nppes WHERE entity_type_code = 2.0 AND provider_business_practice_location_address_city_name='NASHVILLE')
--LIMIT 5
"""
query = """
SELECT *
FROM profile
LIMIT 5
"""

with sqlite3.connect('data/hopteam.sqlite') as db:
    ref_sqlite = pd.read_sql(query, db)   


In [84]:
ref_sqlite

,npi,taxonomy_code,entity_type_code,provider_organization_name_(legal_business_name),provider_last_name_(legal_name),provider_first_name,provider_middle_name,provider_name_prefix_text,provider_name_suffix_text,provider_credential_text,provider_first_line_business_practice_location_address,provider_second_line_business_practice_location_address,provider_business_practice_location_address_city_name,provider_business_practice_location_address_state_name,provider_business_practice_location_address_postal_code
0,1679576722,207X00000X,1.0,None,WIEBE,DAVID,A,None,None,M.D.,3500 CENTRAL AVE,None,KEARNEY,NE,68847
1,1588667638,207RC0000X,1.0,None,PILCHER,WILLIAM,C,DR.,None,MD,1824 KING STREET,SUITE 300,JACKSONVILLE,FL,32204
2,1497758544,251G00000X,2.0,"CUMBERLAND COUNTY HOSPITAL SYSTEM, INC",None,None,None,None,None,None,3418 VILLAGE DR,None,FAYETTEVILLE,NC,28304
3,1023011178,251G00000X,2.0,COLLABRIA CARE,None,None,None,None,None,None,414 S JEFFERSON ST,None,NAPA,CA,94559
4,1841293990,231H00000X,1.0,None,WORTSMAN,SUSAN,None,None,None,MA-CCC,425 E 25TH ST,None,NEW YORK,NY,10010


In [82]:
# display tables in a sqlite db
query = """
SELECT name FROM sqlite_schema
WHERE type = 'table' AND name NOT LIKE 'sqlite_%';
"""
with sqlite3.connect('data/hopteam.sqlite') as db:
    ref_sqlite = pd.read_sql(query, db)
ref_sqlite

,name
0,referral_from
1,referral_to
2,referrals
3,profile


In [26]:
# # drop a table from sqlite db
# db = sqlite3.connect('data/hopteam.sqlite')

# #Creating a cursor object using the cursor() method
# cursor = db.cursor()

# #Doping EMPLOYEE table if already exists
# cursor.execute("DROP TABLE nppes")
# print("Table dropped... ")

# #Commit your changes in the database
# db.commit()

# #Closing the connection
# db.close()

Table dropped... 
